In [ ]:
!pip install particle

In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

from analysis_variables import *

import myPIDselector

import pandas as pd

from particle import Particle

In [ ]:
start = time.time()

# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

##josie laptop 
#topdir = "/Users/josieswann/Desktop/important documents"

# On Bellis' laptop
#topdir = '/home/bellis/babar_data/bnv_plambda/'

# At Bellis' home
topdir = '/home/bellis/babar_data/bnv_plambda'

# On Bellis' laptop
#topdir = './'

filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")
IS_MC=True


#'''
# Collision data
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Data_Only_Run_1_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'

start = time.time()

data_collision = ak.from_parquet(filename)
#data_collision = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")

print(type(data_collision))
#'''


In [ ]:
### information about cross section --> what we'll use to calculate scaling values for histograms 

dataset_information = pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes

In [ ]:
sp = data['spmode']

np.unique(sp.to_list())

In [ ]:
region_definitions

In [ ]:
# Make our histograms
all_hists = bat.create_empty_histograms(hist_defs)

bkg_spmodes = ['998', '1005', '1235', '1237', '3981']
sig_spmodes = ['-999']

spmodes = bkg_spmodes + sig_spmodes

weights = {}
for sp in spmodes:
    weights[sp] = bat.scaling_value(int(sp), dataset_information=dataset_information, cs_data=cs_data, plot=False, verbose=False)
    #weights[sp] = 1

### bat.scaling_value is in Babar_analysis_tools.py 

# Scale the signal higher
weights['-999'] = 1000
weights['0'] = 1

print(weights)
print()
print(spmodes)

In [ ]:
# Pull out some SP mode
spmode = '1005'
#spmode='998'
#spmode='991'
#spmode='-999'
#spmode='0'
#mask_sp = data['spmode'] == spmode

mask_sp = (data['spmode'] == '998') | (data['spmode'] == '1005')


# Collision data
#mask_sp = data['spmode']=='0'

# Make a subset
data_sp = data[mask_sp]

# For data we need to account for blinding
if IS_MC is False:
    nB = data_sp['nB']
    Bp3 = data_sp['Bp3']
    nBp3 = ak.num(Bp3)
    
    mask_to_account_for_blinding = nB == nBp3
    print(len(nB[mask_to_account_for_blinding]), len(nB[~mask_to_account_for_blinding]))
    
    data_sp = data_sp[mask_to_account_for_blinding]

# Use only the data in the fitting region
fit_mask = bat.get_fit_mask(data_sp, region_definitions)

# Make the plot
mes =    ak.flatten(data_sp['BpostFitMes'][fit_mask])
DeltaE = ak.flatten(data_sp['BpostFitDeltaE'][fit_mask])

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
all_hists = bat.create_empty_histograms(hist_defs)
x = ak.flatten(data_sp['Lambda0_unc_Mass'][fit_mask])
weight = weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();

In [ ]:
def dump_truth_information(data, n=0, verbose=0):
    
    if verbose:
        print("The MC and tracks for the first entry")
        idx = data['TRKMCIdx'][n]
        mclund = data['mcLund'][n]

        for i,id in enumerate(idx):
            print(f"{i:2d}  {id:4d}   {mclund[id]}")
    print()

    return 1

##########################################################################
# Pull out some SP mode
#spmode = '1005'
spmode='998'
#spmode='991'
#spmode='-999'
#spmode='0'
mask_sp = data['spmode'] == spmode

dump_truth_information(data[mask_sp],n=10,verbose=1)

In [ ]:
data.fields

In [ ]:
n = 0
print(data[n]['mcLen'])
print(data[n]['mothIdx'])
print(data[n]['mcLund'])
print(data[n]['dauLen'])
print(data[n]['dauIdx'])

idxs_printed = []

for i in range(data[n]['mcLen']):

    mothIdx,mcLund,dauLen,dauIdx = data[n]['mothIdx'][i], data[n]['mcLund'][i], data[n]['dauLen'][i], data[n]['dauIdx'][i]
    
    name = "NONE"
    try:
        name = Particle.from_pdgid(mcLund).name
    except:
        name = "NONE"

    if i not in idxs_printed:
        print(f"{i:3d}  mothIdx:{mothIdx:3d}  mcLund:{mcLund:6d} {name:12s}   ndau: {dauLen:2d}  dauIdx: {dauIdx}")
        if dauLen>0:
            for j in range(dauLen):

                mothIdx2,mcLund2,dauLen2,dauIdx2 = data[n]['mothIdx'][j+dauIdx], data[n]['mcLund'][j+dauIdx], data[n]['dauLen'][j+dauIdx], data[n]['dauIdx'][j+dauIdx]

                try:
                    name2 = Particle.from_pdgid(mcLund2).name
                except:
                    name2 = "NONE"

                if j+dauIdx not in idxs_printed:
                    print(f"\t{j+dauIdx:3d}  mothIdx:{mothIdx2:3d}  mcLund:{mcLund2:6d} {name2:12s}   ndau: {dauLen2:2d}  dauIdx: {dauIdx2}")
                    idxs_printed.append(j+dauIdx)
            
        idxs_printed.append(i)

In [ ]:
#Particle.from_pdgid(10022)
def print_gen_particle(data, i=0, idxs_printed=None):
    mothIdx,mcLund,dauLen,dauIdx = data['mothIdx'][i], data['mcLund'][i], data['dauLen'][i], data['dauIdx'][i]
    
    name = "NONE"
    try:
        name = Particle.from_pdgid(mcLund).name
    except:
        name = "NONE"

    if i not in idxs_printed:
        print(f"{i:3d}  mothIdx:{mothIdx:3d}  mcLund:{mcLund:6d} {name:12s}   ndau: {dauLen:2d}  dauIdx: {dauIdx}")

    return dauIdx

############################
n = 0
particles_printed = []
print_gen_particle(data[n], i=0, idxs_printed=particles_printed)

In [ ]:
#data[0]

In [ ]:
? Particle.from_pdgid

In [ ]:
a = Particle.from_pdgid(2122)
print(type(a))

print(a.name)

a